In [1]:
from models.Detector import Detector
import os
from tqdm import tqdm
import pandas as pd

weights_path = 'models/weights/'

model = Detector(path_to_weights=weights_path, path_to_tmp='./tmp_files', weights_name='detector_weights_v2.pt')

In [2]:
path_to_images = '../dataset'

image_paths = [os.path.join(path_to_images, filename) for filename in os.listdir(path_to_images) if filename.endswith('.jpg')]

In [3]:
predicts = []
for img in tqdm(image_paths):
    predicts += model.predict([img])

100%|██████████| 1162/1162 [03:02<00:00,  6.36it/s]


In [5]:
def format_output(predicts):
    formated_predicts = {
            'filename' : [],
            'class_id' : [],
            'rel_x' : [],
            'rel_y' : [],
            'width' : [],
            'height' : []
        }

    classes_map = {
        "adj": 0,
        "int": 1,
        "geo": 2,
        "pro": 3,
        "non": 4,
    }
    
    for item in predicts:
        image_name = item['source_img_path'].split('/')[-1]
        for i, cls in enumerate(item['names']):
            print(cls)
            parsed_coords = item['xywhn'].tolist()

            formated_predicts['filename'].append(image_name)
            formated_predicts['class_id'].append(classes_map[cls])
            formated_predicts['rel_x'].append(parsed_coords[i][0])
            formated_predicts['rel_y'].append(parsed_coords[i][1])
            formated_predicts['width'].append(parsed_coords[i][2])
            formated_predicts['height'].append(parsed_coords[i][3])
            
    return formated_predicts

In [6]:
pd.DataFrame(format_output(predicts)).to_csv('submission.csv', index=False, sep=';')

non
geo
geo
int
int
pro
int
int
int
int
adj
adj
adj
adj
int
geo
geo
adj
geo
geo
int
int
int
geo
non
geo
geo
geo
int
geo
adj
non
int
non
adj
adj
int
geo
int
int
int
geo
geo
geo
geo
pro
pro
adj
int
adj
adj
adj
adj
adj
pro
geo
int
geo
non
non
int
geo
adj
adj
adj
adj
adj
geo
int
int
int
int
geo
geo
pro
pro
geo
adj
adj
geo
geo
int
adj
adj
adj
adj
adj
adj
adj
adj
pro
adj
pro
geo
adj
int
int
geo
adj
geo
pro
geo
geo
geo
geo
geo
adj
int
int
geo
geo
geo
geo
geo
non
geo
geo
adj
adj
adj
geo
adj
adj
geo
geo
pro
geo
geo
geo
geo
geo
adj
adj
adj
adj
adj
adj
adj
geo
adj
adj
adj
pro
adj
adj
geo
adj
adj
geo
geo
geo
geo
adj
geo
adj
adj
adj
adj
adj
geo
adj
geo
geo
adj
geo
int
adj
non
int
int
geo
adj
non
pro
adj
geo
adj
geo
geo
geo
pro
geo
geo
geo
geo
adj
adj
adj
geo
adj
adj
int
adj
int
geo
adj
pro
adj
adj
pro
geo
int
adj
adj
pro
adj
adj
adj
adj
adj
adj
int
int
int
adj
adj
non
int
adj
adj
adj
geo
adj
adj
pro
adj
adj
adj
adj
adj
adj
adj
adj
geo
pro
adj
adj
adj
adj
geo
adj
adj
adj
adj
geo
non
non
geo
non
non


AttributeError: 'dict' object has no attribute 'to_csv'